# Layer Groups

The REST API allows you to create and modify layer groups in GeoServer.

> **Note:** <br> 
> You can find the official example at [https://docs.geoserver.org/2.25.x/en/user/rest/layergroups.html](https://docs.geoserver.org/2.25.x/en/user/rest/layergroups.html)

## Setup

### Imports

First, we need to import the necessary modules and classes.

In [1]:
from pathlib import Path
from geoserver import GeoServer

### GeoServer Connection

Connect to the running GeoServer instance and create a workspace and a store.

In [2]:
# Setup the geoserver instance
geoserver = GeoServer(
    service_url="http://localhost:8080/geoserver",
    username="admin",
    password="geoserver",
)

Let's clean up existing layer groups before we start.

In [3]:
if geoserver.workspace_exists("demo"):
    geoserver.delete_workspace("demo", recurse=True)

geoserver.create_workspace(body="<workspace><name>demo</name></workspace>")

'Created'

### Config

We'll set up the configuration for the notebook:

In [4]:
# Directory containing sample data
EXAMPLES_DIR = Path("../examples")
assert EXAMPLES_DIR.exists(), f"The directory {EXAMPLES_DIR} does not exist."

## Listing layer groups

To list all layer groups, use the `get_layer_groups` method.

In [5]:
geoserver.get_layer_groups(workspace="demo")

{'layerGroups': ''}

## Creating a layer group

To create a layer group, use the `create_layer_group` method.

First, make sure you have setup the layers you want to include in the layer group.

In [6]:
# Add a coverage store
file_path = EXAMPLES_DIR / "rasters" / "raster1.tif"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"

geoserver.upload_coverage_store(file=file_path, workspace="demo", format="geotiff")


# Add a vector store
file_path = EXAMPLES_DIR / "vectors" / "states.shp"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"

geoserver.upload_data_store(file=file_path, workspace="demo", format="shp")

'Created'

In [7]:
# Using XML format
body = """
<layerGroup>
  <name>my_group</name>
  <layers>
    <layer>states</layer>
    <layer>raster1</layer>
  </layers>
  <styles>
    <style>polygon</style>
    <style>raster1</style>
  </styles>
</layerGroup>
"""

geoserver.create_layer_group(body=body, workspace="demo")

'Created'

## Retrieving a layer group

To retrieve a layer group, use the `get_layer_group` method.

In [8]:
geoserver.get_layer_group(layer_group="my_group", workspace="demo")

{'layerGroup': {'name': 'my_group',
  'mode': 'SINGLE',
  'workspace': {'name': 'demo'},
  'publishables': {'published': [{'@type': 'layer',
     'name': 'demo:states',
     'href': 'http://localhost:8080/geoserver/rest/workspaces/demo/layers/states.json'},
    {'@type': 'layer',
     'name': 'demo:raster1',
     'href': 'http://localhost:8080/geoserver/rest/workspaces/demo/layers/raster1.json'}]},
  'styles': {'style': [{'name': 'polygon',
     'href': 'http://localhost:8080/geoserver/rest/styles/polygon.json'},
    '']},
  'bounds': {'minx': -124.73142200000001,
   'maxx': 1047118.0206637613,
   'miny': 24.955967,
   'maxy': 4562818.101469008,
   'crs': {'@class': 'org.geotools.referencing.crs.DefaultEngineeringCRS',
    '$': 'EPSG:404000'}},
  'dateCreated': '2024-06-05 22:59:34.930 UTC'}}

## Changing a layer group

To update a layer group, use the `update_layer_group` method.

In [9]:
# Using XML format
body = """
<layerGroup>
    <name>my_group</name>
    <layers>
        <layer>states</layer>
    </layers>
    <styles>
        <style>polygon</style>
    </styles>
</layerGroup>
"""

geoserver.update_layer_group(body=body, layer_group="my_group", workspace="demo")

'Updated'

## Removing a layer group

To remove a layer group, use the `delete_layer_group` method.

In [10]:
geoserver.delete_layer_group(layer_group="my_group", workspace="demo")

'Deleted'